In [ ]:
from scipy.sparse import csr_matrix
import numpy as np

# Example function to modify rate_all_items to accept a sparse matrix
def rate_all_items(user_id, sparse_um, similarity_func):
    """
    Rates all items for a given user based on a sparse utility matrix and a similarity function.
    
    Parameters:
        user_id (int): ID of the user to predict ratings for.
        sparse_um (csr_matrix): Sparse utility matrix.
        similarity_func (function): Similarity function, e.g., fast_centered_cosine_sim.
    
    Returns:
        np.array: Predicted ratings for the user across all items.
    """
    user_ratings = sparse_um[user_id]
    all_ratings = []

    for item_idx in range(sparse_um.shape[1]):
        if user_ratings[0, item_idx] == 0:  # Only predict for items not rated
            sim_scores = similarity_func(sparse_um, sparse_um[:, item_idx].transpose())
            # Weighted rating prediction (ignoring user's own rating on the item)
            weighted_sum = sim_scores.dot(sparse_um[:, item_idx].toarray()).item()
            normalizer = np.abs(sim_scores).sum()
            predicted_rating = weighted_sum / normalizer if normalizer != 0 else 0
            all_ratings.append(predicted_rating)
        else:
            all_ratings.append(user_ratings[0, item_idx])

    return np.array(all_ratings)

# Example function to print matrix size
def report_matrix_size(sparse_matrix):
    size_in_mb = sparse_matrix.data.nbytes / (1024 ** 2)
    print(f"Sparse Matrix Size: {size_in_mb:.2f} MB")

# Unit test function
def test_rate_all_items():
    # Create a dense version and a sparse version of the utility matrix for testing
    dense_um = np.random.rand(10, 10)
    sparse_um = csr_matrix(dense_um)

    # Test with user_id 0
    dense_result = rate_all_items_dense(0, dense_um)  # Assuming rate_all_items_dense is the dense version
    sparse_result = rate_all_items(0, sparse_um, fast_centered_cosine_sim)
    
    # Compare results
    assert np.allclose(dense_result, sparse_result, atol=1e-5), "Sparse and dense results differ!"
    print("Test passed: Sparse and dense implementations match.")

test_rate_all_items()